In [1]:
csv_name = 'train_ver2'

In [39]:
# Import necessary stuff
import pandas as pd
import os.path
import numpy as np
from joblib import Parallel, delayed
import time

processors = 4
max_chunksize=10000

In [5]:
# This is my personal data folder. Change to your own
#dir = "/Users/piromast/Documents/Kaggle/SantanderData/"
import os
dir = os.getcwd()
#train_csv = os.path.join(dir,'train_ver2.csv')
client_ids = np.load('../client_ids.npy')
client_index_array = np.load('../' + csv_name + '__client_indices.npy')
client_number = client_ids.shape[0]

In [40]:
import shlex
from subprocess import Popen, PIPE

def get_dataset_lines():
    cmd = "wc -l " + '../'+csv_name + '.csv'
    process = Popen(shlex.split(cmd), stdout=PIPE)
    (output, err) = process.communicate()
    exit_code = process.wait
    train_lines = int(output.split()[0])
    return train_lines

def get_train_reader():
    return pd.read_csv(os.path.join('..',csv_name+'.csv'), chunksize=max_chunksize)

#train_lines = get_dataset_lines()
#print 'train_lines = ',train_lines

train_chunks = int(np.ceil(float(train_lines)/chunksize))
print 'train_chunks = ',train_chunks

train_chunks =  1365


In [5]:
#train_reader = pd.read_csv(train_csv, chunksize=1000)
#train_reader = pd.read_csv(train_csv, iterator=True, chunksize=1000)
#
def get_client_ids():
    client_ids = np.array([],dtype='int')
    for i, chunk in enumerate(train_reader):
        client_ids = np.union1d(client_ids,chunk.ncodpers.unique())
    return client_ids

def create_client_ids_array():
    client_ids = get_client_ids()
    #client_ids.shape
    np.save('client_ids',client_ids)
    
#create_client_ids_array()


In [6]:
class train_chunker(object):
    """
    """
    def __init__(self,chunksize):
        self.chunksize     = chunksize
        self.csv_path      = os.path.join(dir,'train_ver2.csv')
        self.client_ids    = np.load('client_ids.npy')
        self.client_number = self.client_ids.shape[0]

        self.chunks = int(np.ceil(float(self.client_number)/self.chunksize))
        print(str(self.chunks) + ' chunks')
        self.reader = pd.read_csv(self.csv_path, chunksize=self.chunksize)


In [42]:
train_reader = get_train_reader()

def find_client_index(ncodpers):
    return np.argwhere(client_ids == ncodpers)

def create_dataset_client_index_array():

    train_data_client_indices = np.array([],dtype='int')
    #train_data_client_indices = np.full((client_number),np.NaN,dtype='int')
    j = 0

    # measure process time
    import time
    t0 = time.clock()

    for i, chunk in enumerate(train_reader):
        if i%100==0:
            proc_time = time.clock() - t0
            print('chunk ' + str(i) + ' from ' + str(train_chunks) + '. (' + str(proc_time) + ' secs)')

        chunk_size = len(chunk)
        chunk_ncodpers = chunk.ncodpers
        #print chunk_ncodpers[0]
        stdout = Parallel(n_jobs=processors)\
        (delayed(find_client_index)(ncodpers) for ncodpers in chunk_ncodpers)

        chunk_indices = np.full((chunk_size),np.NaN)
        for i, indice in enumerate(stdout):
            chunk_indices[i]= indice

        train_data_client_indices = np.concatenate((train_data_client_indices, chunk_indices))
        #    train_data_client_indices[i*chunksize:(i+1)*chunksize] = chunk_indices

    np.save(csv_name + '__client_indices',train_data_client_indices)

create_dataset_client_index_array()

chunk 0 from 1365. (0.065743 secs)
chunk 100 from 1365. (120.494977 secs)
chunk 200 from 1365. (248.707856 secs)
chunk 300 from 1365. (394.746841 secs)
chunk 400 from 1365. (543.454813 secs)
chunk 500 from 1365. (698.321827 secs)
chunk 600 from 1365. (861.118067 secs)
chunk 700 from 1365. (1025.78539 secs)
chunk 800 from 1365. (1209.056001 secs)
chunk 900 from 1365. (1400.426257 secs)
chunk 1000 from 1365. (1596.30563 secs)
chunk 1100 from 1365. (1790.651775 secs)
chunk 1200 from 1365. (2007.253984 secs)
chunk 1300 from 1365. (2227.411476 secs)


In [ ]:
client_ids[540143]

In [32]:
train_reader = pd.read_csv(os.path.join('..',csv_name+'.csv'), chunksize=chunksize)

for i, chunk in enumerate(train_reader):
    column_ids = chunk.keys()
#            print vars(this_chunk).iteritems()
    product_ids = [s for s in column_ids if "ult1" in s]
    break

In [34]:
train_reader = pd.read_csv(os.path.join('..',csv_name+'.csv'), chunksize=chunksize)

def create_utility_matrix():
    product_number = 10

    utility_matrix = np.zeros((client_number,product_number),dtype='int')
    

    # measure process time
    t0 = time.clock()

    j = 0
    for i, chunk in enumerate(train_reader):
        if i%100==0:
            proc_time = time.clock() - t0
            print('chunk ' + str(i) + ' from ' + str(train_chunks) + '. (' + str(proc_time) + ' secs)')

        chunk_len = len(chunk)
#        print len(chunk)
#        print j+chunk_len
        chunk_client_indices = client_index_array[j:j+chunk_len]
        j+=chunk_len
#        print chunk_client_indices
    print j
            
create_utility_matrix()

chunk 0 from 1365. (0.058221 secs)
chunk 100 from 1365. (4.991478 secs)
chunk 200 from 1365. (9.577015 secs)
chunk 300 from 1365. (14.114101 secs)
chunk 400 from 1365. (19.0129 secs)
chunk 500 from 1365. (25.072869 secs)
chunk 600 from 1365. (30.472365 secs)
chunk 700 from 1365. (35.971098 secs)
chunk 800 from 1365. (41.771981 secs)
chunk 900 from 1365. (47.168134 secs)
chunk 1000 from 1365. (52.131462 secs)
chunk 1100 from 1365. (57.251175 secs)
chunk 1200 from 1365. (62.122019 secs)
chunk 1300 from 1365. (67.157229 secs)
13647309


In [36]:
print 'train_lines = ',train_lines
print client_index_array.shape

train_lines =  13647310
(13650000,)


In [ ]:
np.array(stdout[:])[0]
print stdout[0][0][0]
print stdout[:][0][0]
print stdout[0][:][0]
print stdout[0][0][:]
chunk_indices = np.full((chunksize),np.NaN)
for i, indice in enumerate(stdout):
    chunk_indices[i]= indice
print chunk_indices.shape

In [ ]:
# Import necessary stuff
import pandas as pd
import os.path
import numpy as np

In [ ]:
train_reader = pd.read_csv(train_csv, chunksize=chunksize)
client_id = client_ids[0]
client_matrix = pd.DataFrame()
for i, chunk in enumerate(train_reader):
    client_lines = chunk.ncodpers == client_id
    #print client_lines == True
    client_matrix = client_matrix.append(chunk.ix[client_lines])    
    isinchunk = np.sum(client_lines)

#    if isinchunk != 0.0:
#        print i, isinchunk,len(client_matrix)
#        #print chunk.ix[client_lines]
#        print client_matrix
#        raw_input()
        #break
client_matrix

In [ ]:
for i, chunk in enumerate(train_reader):
    pass
print i
#    print(chunk.ncodpers.unique().shape)

#data=train_df.get_chunk()
#data.ncodpers.unique()
#for chunk_df in train_df:
#    chunk_data = chunk_df.get_chunk()
#    print chunk_data.shape
#read
#data.ncodpers.unique()

In [ ]:
# Import description table (describing the meaning of each column)
description = read_csv(os.path.join(dir,'column_description.csv'),encoding = 'latin1')
description

In [ ]:
# Select product column names and build indeces range
column_names = data.columns.values
product_names = [(i, s) for i, s in enumerate(column_names) if 'ind_' in s and '_ult1' in s]
product_indeces = [e[0] for e in product_names]
mInd = min(product_indeces)
MInd = max(product_indeces)
product_indeces_range = range(mInd,MInd)

In [ ]:
product_names

In [ ]:
# Build a dataframe only with products
product_df = data.iloc[:,product_indeces_range]

In [ ]:
# Build a Series with number of products and use description as indeces
desclist=description.iloc[product_indeces_range,1].tolist()
num_products = Series(product_df.sum().tolist(),index=desclist)
num_products
